In [1]:
import os
import cv2
from tqdm import tqdm
!pip install kaggle
!kaggle datasets download -d joshmanto/food-waste-detection


Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.callbacks import ProgbarLogger #nice progress bar

# Initialize the ProgbarLogger callback
progbar_logger = ProgbarLogger(count_mode='steps')

# Instantiate the legacy Adam optimizer with your desired parameters
adam_optimizer = legacy.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)

# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Setup train and validation generators
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/food-waste-detection/train-end2/train-end2',  # Update to your training directory path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical') #label is based on the name of the folder

val_generator = val_datagen.flow_from_directory(
    '/kaggle/input/food-waste-detection/val-end/val-end',  # Validation directory path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical') #label is based on the name of the folder

# Load and customize ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(241, activation='softmax')(x)  # adjusted to 241 classes
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train for
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,callbacks=[progbar_logger])


#should we add a dropout regularization term>? 




In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/food-waste-detection/train-end2/train-end2',# directory for training data
        target_size=(100, 100),
        batch_size=32,
        class_mode='sparse',
        subset='training')

# Flow validation images in batches of 32 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        '/kaggle/input/food-waste-detection/val-end/val-end', # directory for validation data
        target_size=(100, 100),
        batch_size=32,
        class_mode='sparse',
        subset='validation')

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(241, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the generators
model.fit(train_generator, epochs=20, validation_data=validation_generator)

Found 136777 images belonging to 241 classes.
Found 4113 images belonging to 241 classes.
Epoch 1/20
4275/4275 [==============================] - 737s 172ms/step - loss: 4.6576 - accuracy: 0.0526 - val_loss: 4.3017 - val_accuracy: 0.0776
Epoch 2/20
4275/4275 [==============================] - 704s 165ms/step - loss: 4.0831 - accuracy: 0.1163 - val_loss: 4.0041 - val_accuracy: 0.1281
Epoch 3/20
4275/4275 [==============================] - 688s 161ms/step - loss: 3.7888 - accuracy: 0.1562 - val_loss: 3.9657 - val_accuracy: 0.1252
Epoch 4/20
4275/4275 [==============================] - 693s 162ms/step - loss: 3.5956 - accuracy: 0.1832 - val_loss: 3.9167 - val_accuracy: 0.1376
Epoch 5/20
4275/4275 [==============================] - 697s 163ms/step - loss: 3.4448 - accuracy: 0.2055 - val_loss: 3.8623 - val_accuracy: 0.1515
Epoch 6/20
4275/4275 [==============================] - 700s 164ms/step - loss: 3.3154 - accuracy: 0.2266 - val_loss: 3.9035 - val_accuracy: 0.1461
Epoch 7/20
4275/4275 [

In [12]:
from tensorflow.keras.models import save_model
#Saving the model
model.save('my_model.keras')

In [46]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('my_model.keras')


In [47]:
import numpy as np
from PIL import Image
import os

# Input directory
input_dir = '/kaggle/input/food-waste-detection/processed_images-20240225T160409Z-001/processed_images'

# List all files in the input directory
files = os.listdir(input_dir)

# Iterate over each image and make predictions
for file in files:
        with Image.open(os.path.join(input_dir, file)) as img:
            # Resize the image to 100x100 pixels
            resized_img = img.resize((100, 100))
            converted_img = resized_img.convert('RGB')
            # Convert the image to a numpy array and normalize the pixel values
            normalized_img = np.array(converted_img) / 255.0
            # Add a new axis to match the expected shape of the model
            input_img = normalized_img[np.newaxis, ...]
            # Make predictions on the image
            prediction = model.predict(input_img)
            print(f"File: {file}, Prediction: {prediction}")

1/1 [==============================] - 0s 62ms/step
File: IMG_0326.jpeg, Prediction: [[2.68198723e-12 2.09419460e-16 1.71121018e-14 3.02492626e-05
  3.29587677e-15 3.56151150e-11 8.30667632e-05 6.42151281e-06
  2.58379051e-09 6.63085942e-08 1.71945222e-10 5.98257069e-11
  5.15021565e-13 1.89657783e-06 1.84418106e-11 8.86264173e-10
  2.58982018e-05 2.51473842e-10 1.04874143e-09 2.36538806e-08
  8.32003835e-08 2.45109703e-07 2.23513155e-07 2.43718273e-06
  5.15395782e-10 4.60467118e-06 1.59092124e-07 9.99192591e-13
  1.88449194e-05 2.35954987e-14 8.47897041e-09 3.65274360e-07
  2.37219933e-06 4.15511067e-07 1.99318393e-07 8.65061942e-14
  2.38453822e-06 2.84059265e-09 1.33822221e-06 1.10918748e-11
  5.25186516e-09 5.34479527e-09 2.70158651e-14 2.88589126e-06
  4.29462001e-14 1.49358402e-06 7.10689627e-17 1.87812001e-15
  2.30001105e-11 8.96008325e-14 3.92523919e-15 5.03836084e-10
  3.34465400e-10 9.01624844e-06 2.19595537e-01 9.77092231e-15
  6.60494450e-08 4.90934262e-08 3.20563322e-06 

In [50]:
# Apply hardmax operation
hardmax_predictions = np.zeros_like(predictions)
hardmax_predictions[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

#print("Predictions:")
#print(predictions)
print("\nHardmax Predictions:")
print(hardmax_predictions)
indices = np.where(hardmax_predictions == 1)[1]

print("Indices with value 1 in each row:")
print(indices)


Hardmax Predictions:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Indices with value 1 in each row:
[203 166  31 166 166 166 166 166 166 166 166 203 166 166 166 166 166 166
 166 203 166 166 203 166 166 166 137 166 166 166 166 166 166 166 203 166
  31 166 166 166 166 166 203 166 203 166 166 166 166 166 166 203 170 166
 203 166 166 166 166 166 166 166 166 166 166 166 166 166 166 166 166 166
 166 203 166  20 166 166 166 166 203 203 166 166  34 166 166 203 203 166
  51 166 166 166 203 203 166 166 166 166 137 166 166 166 166 166 203 166
 203 166 166 166 203 166 203 166 166 166 166 203 166 166 166 166 170 166
 170 166 166 166 166 166 203 203 203 166 203 166   6 166 203 166 166  95
 166 166 166 137 166 166 203 203 166 166 166 166 166 166 166 203 166 166
 166 203 166  31 203  31 166 166 166 166 166 203 166 166 166 170 166 166
 166 166 203 166 166 166 166 166 166 166 166 166 203 203 1

In [68]:
import numpy as np
# Count up how many times each index occurs
counts = np.bincount(indices)

print("Counts for each index:")
for index, count in enumerate(counts):
    print(f"Index {index}: {count} occurrences")

Counts for each index:
Index 0: 0 occurrences
Index 1: 0 occurrences
Index 2: 0 occurrences
Index 3: 0 occurrences
Index 4: 0 occurrences
Index 5: 0 occurrences
Index 6: 1 occurrences
Index 7: 0 occurrences
Index 8: 0 occurrences
Index 9: 0 occurrences
Index 10: 0 occurrences
Index 11: 0 occurrences
Index 12: 0 occurrences
Index 13: 0 occurrences
Index 14: 0 occurrences
Index 15: 0 occurrences
Index 16: 0 occurrences
Index 17: 0 occurrences
Index 18: 0 occurrences
Index 19: 0 occurrences
Index 20: 1 occurrences
Index 21: 0 occurrences
Index 22: 0 occurrences
Index 23: 0 occurrences
Index 24: 0 occurrences
Index 25: 0 occurrences
Index 26: 0 occurrences
Index 27: 0 occurrences
Index 28: 0 occurrences
Index 29: 0 occurrences
Index 30: 0 occurrences
Index 31: 8 occurrences
Index 32: 0 occurrences
Index 33: 0 occurrences
Index 34: 2 occurrences
Index 35: 0 occurrences
Index 36: 0 occurrences
Index 37: 0 occurrences
Index 38: 0 occurrences
Index 39: 0 occurrences
Index 40: 0 occurrences
Ind